In [1]:
import graphlab
products = graphlab.SFrame('../data/amazon_baby.gl/')
print "products.shape=", products.shape
print "products.column_names = ", products.column_names()

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: Z:\WINTEMP\graphlab_server_1477951390.log.0


This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.
products.shape= (183531, 3)
products.column_names =  ['name', 'review', 'rating']


Add column for product sentiment.

In [2]:
#ignore all 3* reviews
products = products[products['rating'] != 3]
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

Add column for word frequency count.

In [3]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

list of words assigned by course we will be using as predictors.

In [4]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

create column for each word in 'selected_words' with frequency count.

In [5]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda x: x[word] if word in x else 0L)

In [6]:
print "products.column_names = ", products.column_names()

products.column_names =  ['name', 'review', 'rating', 'sentiment', 'word_count', 'awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']


split products into test & train data sets for model

In [8]:
train_data,test_data = products.random_split(.8, seed=0)
print "train_data.shape = ", train_data.shape
print "test_data.shape = ", test_data.shape

train_data.shape =  (133448, 16)
test_data.shape =  (33304, 16)


In [9]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.367755     | 0.844269          | 0.843142            |

| 5         | 6        | 1.443828     | 0.844269          | 0.843142            |

| 6         | 7        | 1.529911     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [11]:
#subset rows from products the rows where column 'name' has value 'Baby Trend Diaper Champ'
diaperChampReviews = products[products['name'] == 'Baby Trend Diaper Champ']
print "diaperChampReviews.shape = ", diaperChampReviews.shape

diaperChampReviews.shape =  (298, 16)


In [13]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 3.223477     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 4.774215     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [18]:
#create new column & store predictions for rows containing 'Baby Trend Diaper Champ'.
diaperChampReviews['PredictedDaiperSentiment'] = sentiment_model.predict(diaperChampReviews, output_type='probability')

In [25]:
#now sort the SArray by this new column, row zero will be max.
diaperChampReviews = diaperChampReviews.sort('PredictedDaiperSentiment', ascending=False)
print diaperChampReviews['PredictedDaiperSentiment'][0]

0.999999937267


Q 9 : Which of the following ranges contains the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’, according to the sentiment_model from the IPython Notebook from lecture?
ANSWER = 0.9 to 1.0 [refer working above]

In [26]:
mostPositiveReview = diaperChampReviews['review'][0]
print "\nMost positive review for Baby Trend Daiper Champ using Sentiment model = \n\n", mostPositiveReview


Most positive review for Baby Trend Daiper Champ using Sentiment model = 

Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend's house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn't any adde

In [20]:
#create new column & store predictions for rows containing 'Baby Trend Diaper Champ'.
diaperChampReviews['PredictedDaiperSelectedWords'] = selected_words_model.predict(diaperChampReviews, output_type='probability')

In [28]:
diaperChampReviews.filter_by(mostPositiveReview, 'review')['PredictedDaiperSelectedWords']

dtype: float
Rows: 1
[0.796940851290673]

Q10. Consider the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model 
from the IPython Notebook from lecture. 
Which of the following ranges contains the predicted_sentiment for this review, if we use the selected_words_model to analyze it?

Ans: 0.7 to 0.8